# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## To do
- Tidy up callback display
    - Not every iteration
    - Show meaningful metric like objective score
- Optimise for single wind speed for better initial estimate
    - Power not LCOE
    - Different values for below/at rated power
- Increase fidelity
    - Assess difference in speed of calculation and optimal values
- Loop over wind direction
    - Parallelise using PQDM
- Assess results
    - Sensitivity to wind direction/speed
    - Variation in sensitivity between turbines
    - Increase number of turbines

## Setup

In [ ]:
# import libraries
import logging

import numpy as np
from scipy import optimize

import utils

In [ ]:
# define constants
yaw_scale = 30

## Baseline values

In [ ]:
# run baseline simulation
sim_res_base = utils.run_sim()

In [ ]:
# ensure probabilities (wind direction and speed) total 1
Sector_frequency = sim_res_base.Sector_frequency
if not np.isclose(Sector_frequency.sum(), 1):
    logging.warning(
        f"Sector frequency renormalised as total probability was {Sector_frequency.sum().values}"
    )
    Sector_frequency = Sector_frequency / Sector_frequency.sum()
P = sim_res_base.P
if not np.isclose(P.sum(), 1):
    logging.warning(f"P renormalised as total probability was {P.sum().values}")
    P = P / P.sum()

In [ ]:
# calculate baseline metrics
aep_base, lcoe_base, _ = utils.calc_metrics(
    sim_res=sim_res_base,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)
lcoe_direction_base, _, lcoe_overall_base, _ = utils.aggregate_metrics(
    aep=aep_base, lcoe=lcoe_base, Sector_frequency=Sector_frequency
)

## Optimise across for a single wind direction

In [ ]:
# define constants
wd = 270
yaw_shape = (len(sim_res_base.wt), 1, len(sim_res_base.ws))

In [ ]:
# define objective function
def obj_single(yaw_norm):
    sim_res = utils.run_sim(yaw=yaw_norm.reshape(yaw_shape) * yaw_scale, wd=wd)
    aep, lcoe, _ = utils.calc_metrics(
        sim_res=sim_res,
        sim_res_base=sim_res_base,
        Sector_frequency=Sector_frequency,
        P=P,
    )
    _, _, lcoe_overall, _ = utils.aggregate_metrics(
        aep=aep, lcoe=lcoe, Sector_frequency=Sector_frequency
    )
    obj = (lcoe_overall / lcoe_direction_base.sel(wd=wd)).values.tolist()
    return obj

In [ ]:
# check objective function correct for baseline case
assert np.isclose(obj_single(np.zeros(yaw_shape)), 1)

In [ ]:
# run optimisation with iteration counter display
n = 0


def _callback(x):
    global n
    print(f"--- Iteration {n} ---")
    n += 1


res = optimize.minimize(
    fun=obj_single, x0=np.zeros(yaw_shape).ravel() + 1 / yaw_scale, callback=_callback
)
res